In [5]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent, create_csv_agent
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

True

In [6]:
import pandas as pd
llm=ChatGroq(model='llama-3.1-70b-versatile')
df=pd.read_csv('salaries_2023.csv')

In [11]:
agent=create_pandas_dataframe_agent(llm=llm, df=df, verbose=True, allow_dangerous_code=True)

In [12]:
res=agent.invoke("What is the average salary?")



> Entering new AgentExecutor chain...
Thought: To calculate the average salary, we need to calculate the average of the 'Base_Salary' column in the dataframe.

Action: python_repl_ast
Action Input: `df['Base_Salary'].mean()`90312.16574420367Thought: I now know the final answer
Final Answer: The average salary is 90312.16574420367.

> Finished chain.


In [13]:
print(res)

{'input': 'What is the average salary?', 'output': 'The average salary is 90312.16574420367.'}


In [26]:
agent.invoke("How many rows are there in the database?")



> Entering new AgentExecutor chain...
Thought: To find the number of rows in the dataframe, I should use the shape property of the dataframe.

Action: python_repl_ast
Action Input: df.shape(10291, 8)Action: python_repl_ast
Action Input: df.shape[0]10291Thought: Now that I have the number of rows in the dataframe, I can answer the question.

Final Answer: There are 10291 rows in the database.

> Finished chain.


{'input': 'How many rows are there in the database?',
 'output': 'There are 10291 rows in the database.'}

In [15]:
CSV_PROMPT_PREFIX="""
First set the pandas display options to show all the columns, get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX="""
- **ALWAYS** before giving the Final Answer, try another method. Then reflect on the answers of the two methods you
did and ask yourself if it answers the original question correctly.
If you are not sure, try another method.
FORMAT 4 FIGURES OR MORE WITH COMMAS.
- If the methods tried do not give the same result, reflect and try again until
you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that you are not sure of the answer
- If you are sure of the correct answer, create a beautiful and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got to the answer on
a section that starts with: "\n\n Explanation: \n"
In the explanation, mention the column names that you used to get to the final answer.
"""

In [16]:
QUESTION="Which grade has the highest average base salary, and compare the average female pay vs male pay?"

In [27]:
res=agent.invoke(CSV_PROMPT_PREFIX+QUESTION+CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: First, I need to set the pandas display options to show all the columns and get the column names.

Action: python_repl_ast
Action Input: 
```python
import pandas as pd
pd.set_option('display.max_columns', None)
print(df.columns)
```Index(['Department', 'Department_Name', 'Division', 'Gender', 'Base_Salary',
       'Overtime_Pay', 'Longevity_Pay', 'Grade'],
      dtype='object')
Thought:  Now that we have the column names, we can proceed to answer the question. We need to find the grade with the highest average base salary and compare the average female pay vs male pay.

Action: python_repl_ast
Action Input: 
```python
# Group by 'Grade' and calculate the average 'Base_Salary'
avg_base_salary = df.groupby('Grade')['Base_Salary'].mean().sort_values(ascending=False)
print(avg_base_salary)

# Group by 'Gender' and calculate the average 'Base_Salary'
avg_base_salary_gender = df.groupby('Gender')['Base_Salary'].mean()
print(avg_base_salary_gen

In [28]:
res['output']

"The grade with the highest average base salary is 'EX0' with an average base salary of $292,000.00, and the average female pay is $87,497.50 and the average male pay is $92,382.93.\n\nExplanation:\nTo get to the answer, we used the following column names:\n\n- 'Grade': This column was used to group by the different grades and calculate the average base salary for each grade.\n- 'Base_Salary': This column was used to calculate the average base salary for each grade and for each gender.\n- 'Gender': This column was used to group by the different genders and calculate the average base salary for each gender.\n\nWe used the `groupby` function to group by 'Grade' and 'Gender', and the `mean` function to calculate the average 'Base_Salary' for each group. We also used the `sort_values` function to sort the average base salaries for each grade in descending order."